In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter
import torchvision.models as models
import torchvision.utils as vutils
from torchvision import datasets
import time
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [4]:
with open('../data/shijing.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

In [5]:
data=''.join(data)

In [6]:
len(data)

41804

In [7]:
len(set(data))

2848

In [8]:
data[1000:1200]

'有巢，維鳩盈之，之子于歸，百兩成之。\n\n《召南・采蘩》\n于以采蘩，于沼于沚，于以用之，公侯之事。\n于以采蘩，于澗之中，于以用之，公侯之宮。\n被之僮僮，夙夜在公，被之祁祁，薄言還歸。\n\n《召南・草蟲》\n喓喓草蟲，趯趯阜螽。未見君子，憂心忡忡；亦既見止，亦既覯止，我心則降。\n陟彼南山，言采其蕨。未見君子，憂心惙惙；亦既見止，亦既覯止，我心則說。\n陟彼南山，言采其薇。未見君子，我心傷悲；亦既見止，亦既覯'

In [9]:
chars = list(set(data))

In [10]:
# data I/O
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 41804 characters, 2848 unique.


In [11]:
# hyperparameters
hidden_size = 128 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for

In [12]:
X_train = np.zeros((len(data), len(chars)))

char_id = np.array([chars.index(c) for c in data])

X_train[np.arange(len(X_train)), char_id] = 1

y_train = np.roll(char_id,-1)

In [13]:
X_train.shape

(41804, 2848)

In [14]:
y_train.shape

(41804,)

In [15]:
vocab_size

2848

In [16]:
len(X_train)//seq_length * seq_length

41800

In [17]:
def get_batch(X_train=X_train, y_train=y_train, seq_length=seq_length):
    #X_ids = list(range(len(X)))
    #random.shuffle(X_ids)    
    #X = X[X_ids]
    #y = y[X_ids]
    #truncate_id = len(X_train)//seq_length * seq_length
    X = torch.from_numpy(X_train).float()
    y = torch.from_numpy(y_train).long()
    for i in range(0, len(X), seq_length):   
        id_stop = i+seq_length if i+seq_length < len(X) else len(X)
        yield([X[i:id_stop], y[i:id_stop]])

In [18]:
def sample_chars(X_seed, h_prev, length=20):
    #for p in rnn.parameters():
    #    p.requires_grad = False
    X_next = X_seed
    results = []
    with torch.no_grad():
        for i in range(length):        
            y_score, h_prev = rnn(X_next.view(1,1,-1), h_prev)
            y_prob = nn.Softmax(0)(y_score.view(-1)).detach().numpy()
            y_pred = np.random.choice(chars,1, p=y_prob).item()
            results.append(y_pred)
            X_next = torch.zeros_like(X_seed)
            X_next[chars.index(y_pred)] = 1
            #print(f'{i} th char:{y_pred}')|
    #for p in rnn.parameters():
    #    p.requires_grad = True
    return ''.join(results)

In [19]:
class nn_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, X, hidden):
        _, hidden = self.lstm(X, hidden)
        output = self.out(hidden[0])
        return output, hidden
    
    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size)
               )

In [20]:
vocab_size

2848

In [21]:
hidden_size

128

In [34]:
rnn = nn_LSTM(vocab_size, hidden_size, vocab_size)

In [35]:
loss_fn = nn.CrossEntropyLoss()

In [36]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.005)

In [27]:
X_batch, _ = get_batch(X_train, y_train, seq_length).__next__()

In [30]:
sample_chars(X_batch[0], rnn.initHidden(), 100)

'祖祀蘭震匐茅忘蚤杞畛置鼎僭畜跂蒙戩樛匹飫鄭沱慍捄亨耿瑲椅禴鈞饁營騤荇棗楅番砠欲丱主漼罪凝噳輊瞍淠噦南館濩間鶉飽稼彭享闥慝留酲憎恆蓍羜瑲遑爪塈仲怛殲扤迎黍枸宗域投惔寺乃門場控尸隤禎池鰷・彤鷮北故嫁初觥頃'

In [31]:
def train(X_batch, y_batch):
    h_prev = rnn.initHidden()
    optimizer.zero_grad()
    batch_loss = torch.tensor(0, dtype=torch.float)
    
    for i in range(len(X_batch)):
        y_score, h_prev = rnn(X_batch[i].view(1,1,-1), h_prev)
        loss = loss_fn(y_score.view(1,-1), y_batch[i].view(1))
        batch_loss += loss
    batch_loss.backward()
    # Add parameters' gradients to their values, multiplied by learning rate    
    optimizer.step()

    return y_score, batch_loss/len(X_batch)

In [43]:
writer = SummaryWriter(f'logs/lstm1_{time.strftime("%Y%m%d-%H%M%S")}')

In [46]:
all_losses = []
print_every = 100
for epoch in range(100):    
    for batch in get_batch(X_train, y_train, seq_length):
        X_batch, y_batch = batch
        _, batch_loss = train(X_batch, y_batch)
        all_losses.append(batch_loss.item())
        if len(all_losses)%print_every==1:
            print(f'----\nRunning Avg Loss:{np.mean(all_losses[-print_every:])} at iter: {len(all_losses)}\n----')
            writer.add_scalar('loss', np.mean(all_losses[-100:]), len(all_losses))
            print(sample_chars(X_batch[0], rnn.initHidden(), 200))

----
Running Avg Loss:0.7671116590499878 at iter: 1
----
關南關，載號載呶。
鹿鳴露兮，予伯之。
睆彼都人，薄采其藻。
有兔斯首，燔之炙之。
惠而女馬，言綸之後，君子來朝。
奄有飛卉，維水湯湯，上入百神，神英簡女，鳴用棘服。
允召公育，以介我與。
三命之孫，作邑于煁。天子萬年，不我侮；命不見，如南賓之，人相憂矣，言韔之孫，四國是極。
南有樛木，葛藟縈之，有三百心，是用不幾，不我不見。
君子有酒，酌言大旅，女心忉遑，殆既大邦，作家之積。
譽處夭華，夫盈不詹
----
Running Avg Loss:1.389125325679779 at iter: 101
----
如洵，既見求子，不我亨之！
匪卷者朅，亦可以簸，而微求之。

《邶風・承祀》
野有鵲之，采采芣苢。
君子所憇，我心傷悄；殽樂君子，逝不我思！
中谷有麥，朱芾斯戾，薄言捋之。

《邶風・采綠》
葛有襛矣，乘其實羽，綠水黃黃。菽藟召伯！
弭設騰泄五燕邶，振公君子，于以奠之。君子有喪，何求沼之？嗟女懷人，蘊踐發兮。江之速之，伐風發東？跂彼東門，播厥兄弟，于嗟乎騶虞！
江有汜，君子有食，桃之夭夭，莫敢遑處


KeyboardInterrupt: 

In [47]:
print(sample_chars(X_batch[2], rnn.initHidden(), 10000))

。
參差荇菜，磬浚之音，，惄如玉女；何國之孫。不盈頃也！

《邶風・瞻彼襛矣》
出宿于泲，詵日居體，如惔如育，寤穉善朝，誰亟君子，不我亨也。匪樂不述，永日方思！
戎辟匪曀，惠而不遄，
何以亡兮，于彼女子，不宜有觼！
居與子惠，歸哉歸哉，人我乎極，送我乎淇之賢。
充耳苦夭，于我不述！胡得七人，歸遠無父。
未見君子，不忮不求，用求之子，莫知我後！

《邶風・凱風》我諸且！俾我其陳，棘心則說，亦釣維絲。士曰何其？心之憂矣！
終風且曀，不日不曀，在用之子，有也不愚。寔命不來，靜言自之，與子偕心。

《邶風・終風》
綠衣之裳，其實其私。我其君子，不如景之。

《邶風・騶虞》
雄雉》摽有凱，匪澣孟子，送我不述！
終風且曀，不屑虺也，人后無然，女子惠也，徐方不多。申人兄弟，如飛有之；母氏如手，不民不忘，實屢如雲，徂心則降。
溥天之下，何嗟而微。

《邶風・君子于關；諸兄之夢，于嗟洵兮，乃求而無古朝其所！

《邶風・柏舟》
雄雉之茨，棘人日月，奮各有身；爲王爲士，日斯歸？，遠曷云哉！
摽其愬，踊躍用兵，報以求之，庶幾爰處。
有女者曀，秉心則嚏，胡然乎處！
終任且暴，如何人哉！不戢于楚，豈無爲好！
日居月諸，綠衣五總，委蛇委蛇，棘人右遊，何天不思！日居月諸。

《邶風・柏舟》
雄雉》壹葛，悠悠我思，寤言不寐。相林有穴，實維其則矣。

《邶風・終風》
習習谷風，爰白悠關，亦既覯止，不盈有行。我彼行潦，復自夭兮。

《邶風・騶虞》
雄雉于飛，勿翦勿敗，死澗之中，中心不說。
參差荇菜，鳴澣信南，惠而南山之執，于以食之，中猶是敢。
實迺求之，寤用其星，維實之子兮，在室則笑。南有樛木，今孌羊矣，殆其掇之。
昔契樂。我既平羽，常則同行，忉遠于寡人。
嗟我乎騶虞！胡君子偕心。

《邶風・新中》
雄雉于飛，頡之頏之，素絲五紽，居兄無弟。
云何其人，不我思馬。曷有匪也，上乎不敖。胡能有定，云何何求？何以下民，尚如哉！

《邶風・雄雉》
瑳而浥行，與子偕老。諸子有心，兩宴是憲。
宜爾子孫，懷諒兄弟，薄言濯愬！

《邶風・凱風》
敖以據，維葉有衣，執子之遠，不云日也！

《邶風・凱風》
雄雉于飛，不盈頃筐，夙夜維公，懷哉懷懷。
其衾在夏，好氏寤瘉，無我室家，悠我父兮。
罪卷也，柔孫之春，歸哉歸哉！

《邶風・君子苦星》
新羣黃歆，憂心如怒；
風雨在南，頃筐塈之。所謂伊人，靡所與同！
殷其靁，在河衣之，歸哉